<a href="https://colab.research.google.com/github/alexrofail/Inits-Basics-CallBacks/blob/main/convNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch.nn.functional as F
import gzip
import pickle
from torch import tensor
from torch import nn
from torch.utils.data import DataLoader

In [2]:
import pickle, gzip, math, torch, matplotlib as mpl

In [3]:
def get_data():
    with gzip.open('mnist.pkl.gz') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

In [4]:
x_train,y_train,x_valid,y_valid = get_data()

In [8]:
class Dataset():
  def __init__(self,x,y):
    self.x ,self.y = x,y
  
  def __len__(self):
    return len(self.x)

  def __getitem__(self,i):
    return self.x[i], self.y[i]

In [14]:
class DataBunch():
  def __init__(self, train_dl, valid_dl, c=None):
    self.train_dl, self.valid_dl, self.c = train_dl, valid_dl, c

  @property
  def train_ds(self): return self.train_dl.dataset

  @property
  def valid_ds(self): return self.valid_dl.dataset

In [16]:
def get_dls(train_ds, valid_ds, bs, **kwargs):
  return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
          DataLoader(valid_ds, batch_size=bs*2, **kwargs))

In [5]:
def normalize(x,m,s): return (x-m)/s

In [6]:
#export
def normalize_to(train, valid):
  m,s = train.mean(), train.std()
  return normalize(train, m, s), normalize(valid, m,s)

In [9]:
x_train, x_valid = normalize_to(x_train,x_valid)
train_ds, valid_ds = Dataset(x_train,y_valid), Dataset(x_valid, y_valid)

In [11]:
x_train.mean(), x_train.std()

(tensor(2.9752e-08), tensor(1.))

In [19]:
nh, bs = 50, 512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)